In [2]:
import dolfinx.fem as fem
import dolfinx.mesh as mesh
import dolfinx.cpp as cpp
import dolfinx as dlf
from dolfinx.plot import create_vtk_mesh 
from dolfinx import io
import mpi4py.MPI as mpi

import pyvista as pv
import matplotlib.pyplot as plt

import ufl
import gmsh
from dolfinx import geometry

import numpy as np

In [3]:
gmsh.initialize()
membrane = gmsh.model.occ.add_disk(0, 0, 0, 1, 1)
gmsh.model.occ.synchronize()
gdim = 2
status = gmsh.model.addPhysicalGroup(gdim, [membrane], 1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.05)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.05)
gmsh.model.mesh.generate(gdim)
if mpi.COMM_WORLD.rank == 0:
    geometry_data = io.extract_gmsh_geometry(gmsh.model)
    topology_data = io.extract_gmsh_topology_and_markers(gmsh.model)

if mpi.COMM_WORLD.rank == 0:
    # Extract the cell type and number of nodes per cell and broadcast
    # it to the other processors
    gmsh_cell_type = list(topology_data.keys())[0]
    properties = gmsh.model.mesh.getElementProperties(gmsh_cell_type)
    name, dim, order, num_nodes, local_coords, _ = properties
    cells = topology_data[gmsh_cell_type]["topology"]
    cell_id, num_nodes = mpi.COMM_WORLD.bcast([gmsh_cell_type, num_nodes],root=0)
else:
    cell_id, num_nodes = mpi.COMM_WORLD.bcast([None, None], root=0)
    cells, geometry_data = np.empty([0, num_nodes]), np.empty([0, gdim])

ufl_domain = io.ufl_mesh_from_gmsh(cell_id, gdim)
gmsh_cell_perm = io.cell_perm_gmsh(
    cpp.mesh.to_type(str(ufl_domain.ufl_cell())), num_nodes)
cells = cells[:, gmsh_cell_perm]
domain = mesh.create_mesh(mpi.COMM_WORLD, cells, geometry_data[:, :gdim],
                          ufl_domain)


Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Done meshing 1D (Wall 0.000357s, CPU 0.000332s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.128793s, CPU 0.130924s)
Info    : 1550 nodes 3099 elements


In [4]:
space = fem.FunctionSpace(domain, ('CG', 1))
x = ufl.SpatialCoordinate(domain)

In [5]:
def bound_cond(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2),1)

bounadary_dofs = fem.locate_dofs_geometrical(space,bound_cond)
print(bound_cond)
bcs= fem.dirichletbc(np.float64(0.0),bounadary_dofs,space)

<function bound_cond at 0x7f6c043bba60>


In [6]:
Dot = ufl.dot
Grad = ufl.grad
dx = ufl.dx

In [7]:
u = ufl.TrialFunction(space)
v = ufl.TestFunction(space)

beta = fem.Constant(domain, np.float64(12))
r0 = fem.Constant(domain, np.float64(0.3))
p = 4 * ufl.exp(-beta**2 * (x[0]**2 + (x[1] - r0)**2))

a = Dot(Grad(u), Grad(v)) * dx
L = p * v * dx

problem = fem.petsc.LinearProblem(a,L,bcs=[bcs])

In [8]:
uh = problem.solve()

In [9]:
Q= fem.FunctionSpace(domain,('CG',5))
expression= fem.Expression(p,Q.element.interpolation_points)
pressure = fem.Function(Q)
pressure.interpolate(expression)

In [10]:
pv.set_jupyter_backend('pythreejs')
# pv.set_jupyter_backend('ipygany')
grid_1 = pv.UnstructuredGrid(*create_vtk_mesh(space))
grid_1.point_data['f']=uh.x.array
warped= grid_1.warp_by_scalar('f',factor=10)
plot_1 = pv.Plotter(window_size=(800,400),shape=(1,2))
ax_11= plot_1.subplot(0,0)
plot_1.add_mesh(grid_1)
ax_12= plot_1.subplot(0,1)
plot_1.add_mesh(warped,show_scalar_bar=False)
plot_1.show()

GridspecLayout(children=(Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.25, position…

In [11]:
pv.set_jupyter_backend('ipygany')
plot_2 = pv.Plotter(window_size=(800,400))
grid_2=pv.UnstructuredGrid(*create_vtk_mesh(Q))
grid_2.point_data['f']=pressure.x.array.real
warped_2= grid_2.warp_by_scalar('f',factor=0.1)
plot_2.add_mesh(warped_2,show_scalar_bar=True)
# plot_2.show()

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x4b65d60) at 0x7f6c01449580>

In [25]:
tol = 0.01
y = np.linspace(-1 + tol, 1 - tol, 101)
x=np.ones_like(y)*0.1

view_line1 = np.zeros((101,3))
view_line1[:,0] = 0
view_line1[:,1] = y

view_line2= np.zeros((101,3))
view_line2[:,0]=0.1
view_line2[:,1] = y

In [23]:
def make_plot(points):
    # Collisions:
    geometry_box = geometry.BoundingBoxTree(domain, domain.topology.dim)

    cell_coodrinates = geometry.compute_collisions(geometry_box, points)
    colling_cells = geometry.compute_colliding_cells(domain, cell_coodrinates,points)
    
    u_values = []
    v_values = []
    cells = []
    points_onprocessor = []
    
    for i, point in enumerate(points):
        if len(colling_cells.links(i)) > 0:
            points_onprocessor.append(point)
            cells.append(colling_cells.links(i)[0])


In [34]:
colling_cells.links(2)

array([552], dtype=int32)

In [ ]:
fig = plt.figure()
plt.plot(point[:,1], 50*u_values, "k", linewidth=2, label="Deflection ($\\times 50$)")
plt.plot(point[:, 1], p_values, "b--", linewidth = 2, label="Load")
plt.grid(True)
plt.xlabel("y")
plt.legend()
# If run in parallel as a python file, we save a plot per processor
plt.savefig(f"membrane_rank{MPI.COMM_WORLD.rank:d}.png")

NameError: name 'points_on_proc' is not defined

<Figure size 432x288 with 0 Axes>